<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/ORS%E3%83%81%E3%83%88%E3%82%BB%E3%83%94%E3%82%A22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ============================================
# チトセピア専用（差し替えは ORS_API_KEY 入力のみ）
# 非円形アイソクローン(15/30/45分) → GeoJSON/KML
# ドーナツ(非重複リング)KML → Zip化 → ダウンロード（Colab）
# ============================================

# 依存ライブラリ
!pip -q install openrouteservice shapely geopandas fiona simplekml pyproj

# ---- ここだけ差し替えればOK（入力方式） ----
# ※ 直書きせず、実行時に貼り付けてください
ORS_API_KEY = input("ORS_API_KEY を貼り付けて Enter: ").strip()
# -------------------------------------------

# チトセピア用プリセット
CENTER_NAME = "チトセピア"
CENTER_COORD = [32.7881, 129.8562]        # [lat, lon]
PROFILE = "driving-car"                   # 車到達圏
RANGES_MIN = [15, 30, 45]                 # 1次/2次/3次（分）
SMOOTH_TOLERANCE = 0.0002                 # Shapely簡素化（軽量化/視認性）
SMOOTHING_API = 5                         # ORS滑らかし(0-100)
AUTO_DOWNLOAD = True                      # 生成Zipを自動DLする

# --- インポート ---
import os, shutil
from datetime import datetime
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely import union_all
import openrouteservice as ors
import simplekml
from google.colab import files

# バリデーション（プレースホルダーを弾く：YOUR_ORS_API_KEY はNG）
def _assert(cond, msg):
    if not cond:
        raise ValueError(msg)
_assert(bool(ORS_API_KEY) and ORS_API_KEY != "YOUR_ORS_API_KEY", "ORS_API_KEY を設定してください。")

# キープレビュー（先頭/末尾4文字のみ表示）
print("Key preview:", ORS_API_KEY[:4] + "..." + ORS_API_KEY[-4:])

# ORS 呼び出し
client = ors.Client(key=ORS_API_KEY)
center_lonlat = [CENTER_COORD[1], CENTER_COORD[0]]  # ORSは [lon, lat]

# 例外処理は属性名の差異に耐える形で
try:
    res = client.isochrones(
        locations=[center_lonlat],
        profile=PROFILE,
        range=[m * 60 for m in RANGES_MIN],  # 分→秒
        range_type="time",
        attributes=["area"],                  # total_pop 等はプラン依存
        smoothing=SMOOTHING_API
    )
except Exception as e:
    code = getattr(e, "status", None) or getattr(e, "status_code", None)
    msg  = getattr(e, "message", None) or str(e)
    raise RuntimeError(
        "ORS API エラー: " + (f"HTTP {code} " if code else "") + msg +
        "\n403の主因: キーでIsochrones未許可 / ドメイン・IP制限 / キー無効化 / プレースホルダー使用"
    ) from e

features = res.get("features", [])
_assert(features, "アイソクローン取得に失敗（featuresが空）。座標・APIキー・PROFILEを確認。")

# GeoDataFrame化（15→30→45の順）
features_sorted = sorted(features, key=lambda f: f["properties"]["value"])
gdf = gpd.GeoDataFrame.from_features(features_sorted, crs="EPSG:4326")
gdf["geometry"] = gdf["geometry"].simplify(SMOOTH_TOLERANCE, preserve_topology=True)

# 出力フォルダ
ts = datetime.now().strftime("%Y%m%d_%H%M")
base = f"{CENTER_NAME}_非円形到達圏_{PROFILE}_{ts}"
os.makedirs(base, exist_ok=True)

# レイヤごとに GeoJSON / KML 出力
geojson_paths, kml_paths = [], []

def save_layer(sub_gdf, layer_name):
    gj_path = os.path.join(base, f"{base}_{layer_name}.geojson")
    sub_gdf.to_file(gj_path, driver="GeoJSON", encoding="utf-8")
    geojson_paths.append(gj_path)
    kml = simplekml.Kml()
    geom = sub_gdf.iloc[0].geometry
    polys = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
    folder = kml.newfolder(name=layer_name)
    for poly in polys:
        coords = list(poly.exterior.coords)
        pol = folder.newpolygon(name=layer_name, outerboundaryis=coords)
        pol.style.polystyle.fill = 1
        pol.style.polystyle.outline = 1
    kml_path = os.path.join(base, f"{base}_{layer_name}.kml")
    kml.save(kml_path)
    kml_paths.append(kml_path)

for i, minutes in enumerate(RANGES_MIN, start=1):
    layer_name = f"{i}次_車{minutes}分"
    save_layer(gdf.iloc[[i-1]].copy(), layer_name)

# 中心点KML
kmlp = simplekml.Kml()
p = kmlp.newpoint(name=CENTER_NAME, coords=[(center_lonlat[0], center_lonlat[1])])
p.style.labelstyle.scale = 1.2
p.description = f"{CENTER_NAME}（基点）"
center_kml = os.path.join(base, f"{base}_中心点.kml")
kmlp.save(center_kml)

# ドーナツ（非重複リング）KML作成
donuts_dir = os.path.join(base, "donuts_v2")
os.makedirs(donuts_dir, exist_ok=True)
g1, g2, g3 = gdf.iloc[[0]].copy(), gdf.iloc[[1]].copy(), gdf.iloc[[2]].copy()
inner15, inner30 = union_all(list(g1.geometry.values)), union_all(list(g2.geometry.values))
ring1 = g1.copy()
ring2 = g2.copy(); ring2["geometry"] = ring2.geometry.difference(inner15)  # 30 - 15
ring3 = g3.copy(); ring3["geometry"] = ring3.geometry.difference(inner30)  # 45 - 30

def to_kml(gdf_in, out_path, layer_name):
    kml = simplekml.Kml()
    for geom in gdf_in.geometry:
        if geom.is_empty: continue
        polys = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
        folder = kml.newfolder(name=layer_name)
        for poly in polys:
            coords = list(poly.exterior.coords)
            pol = folder.newpolygon(name=layer_name, outerboundaryis=coords)
            pol.style.polystyle.fill = 1
            pol.style.polystyle.outline = 1
    kml.save(out_path)

donut_kmls = []
p1 = os.path.join(donuts_dir, "1次_車15分_ドーナツ.kml"); to_kml(ring1, p1, "1次_車15分"); donut_kmls.append(p1)
p2 = os.path.join(donuts_dir, "2次_車30分_ドーナツ.kml"); to_kml(ring2, p2, "2次_車30分"); donut_kmls.append(p2)
p3 = os.path.join(donuts_dir, "3次_車45分_ドーナツ.kml"); to_kml(ring3, p3, "3次_車45分"); donut_kmls.append(p3)

# Zip一括出力（通常版/ドーナツ版）
def zip_kml_only(folder, zip_basename, files):
    tmp = f"{zip_basename}_tmp"; os.makedirs(tmp, exist_ok=True)
    for k in files: shutil.copy2(k, os.path.join(tmp, os.path.basename(k)))
    shutil.make_archive(zip_basename, 'zip', tmp); shutil.rmtree(tmp)

def zip_kml_folder(kml_folder, zip_basename):
    if not os.path.isdir(kml_folder): return None
    shutil.make_archive(zip_basename, 'zip', kml_folder)
    return f"{zip_basename}.zip"

zip_normal = f"{CENTER_NAME}_KML_通常版_{ts}"
zip_donut  = f"{CENTER_NAME}_KML_ドーナツ_{ts}"
zip_kml_only(base, zip_normal, kml_paths + [center_kml])
donut_zip_path = zip_kml_folder(donuts_dir, zip_donut)

# 出力表示
print("=== 出力（GeoJSON）==="); [print(p) for p in geojson_paths]
print("\n=== 出力（KML）==="); [print(p) for p in kml_paths]
print("\n中心点（KML）:", center_kml)
print("\n=== ドーナツ（KML）==="); [print(p) for p in donut_kmls]
import os
print("\n=== Zip 出力 ===")
print("通常版KML（Zip）:", os.path.abspath(f"{zip_normal}.zip"))
if donut_zip_path: print("ドーナツ版KML（Zip）:", os.path.abspath(donut_zip_path))

# 自動ダウンロード
if AUTO_DOWNLOAD:
    files.download(f"{zip_normal}.zip")
    if donut_zip_path:
        files.download(donut_zip_path)

print("\n▼My Maps 取込手順")
print("1) My Maps → 新しい地図 → レイヤ 4つ（1次/2次/3次/中心点）")
print("2) 通常版：3次→2次→1次→中心点の順で各KMLをインポート（下から重ねる）")
print("   ドーナツ版：donuts_v2内のKMLを各レイヤに割当（重なり無しで見やすい）")
print("3) 同系色で 3次=薄 / 2次=中 / 1次=濃、線は1-2px")


ORS_API_KEY を貼り付けて Enter: eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImFjNTk2NzQ1ZjllZDRmODI5YmZlN2EzMjQyZWZhZjI0IiwiaCI6Im11cm11cjY0In0=
Key preview: eyJv...In0=
=== 出力（GeoJSON）===
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_1次_車15分.geojson
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_2次_車30分.geojson
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_3次_車45分.geojson

=== 出力（KML）===
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_1次_車15分.kml
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_2次_車30分.kml
チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_3次_車45分.kml

中心点（KML）: チトセピア_非円形到達圏_driving-car_20250912_2317/チトセピア_非円形到達圏_driving-car_20250912_2317_中心点.kml

=== ドーナツ（KML）===
チトセピア_非円形到達圏_driving-car_20250912_2317/donuts_v2/1次_車15分_ドーナツ.kml
チトセピア_非円形到達圏_driving-car_20250912_2317/donuts_v2/2次_車30分_ドーナ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


▼My Maps 取込手順
1) My Maps → 新しい地図 → レイヤ 4つ（1次/2次/3次/中心点）
2) 通常版：3次→2次→1次→中心点の順で各KMLをインポート（下から重ねる）
   ドーナツ版：donuts_v2内のKMLを各レイヤに割当（重なり無しで見やすい）
3) 同系色で 3次=薄 / 2次=中 / 1次=濃、線は1-2px
